In [3]:
print("hello rag")

hello rag


In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    
    return chunks


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load embeddings model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

def create_chroma_db(chunks):
    vector_db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")
    return vector_db


In [17]:
def retrieve_relevant_docs(query, vector_db, k=3):
    docs = vector_db.similarity_search(query, k=k)
    return docs


In [31]:
import ollama

def generate_response(query, retrieved_docs):
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    prompt = f"Assume you are a HR manager. Use the following context to answer the query:\n\n{context}\n\nQuery: {query}"
    
    response = ollama.chat(model="deepseek-r1:8b", messages=[{"role": "user", "content": prompt}])
    return response['message']['content']


In [1]:
pdf_path = "sample_cv.pdf"  # Change this to your PDF file
query = "How would you rate you the candidate out of 10"  # Change this to your query

# Step 1: Load and Process PDF
chunks = load_and_split_pdf(pdf_path)

# Step 2: Store in ChromaDB
vector_db = create_chroma_db(chunks)

# Step 3: Retrieve Relevant Chunks
retrieved_docs = retrieve_relevant_docs(query, vector_db)

# Step 4: Generate Answer
response = generate_response(query, retrieved_docs)

print("Response:", response)


NameError: name 'load_and_split_pdf' is not defined